# NEAK loader notebook alkalmazás v2

A NEAK honlapról letölthető Access adatbázisokat tölti be egy SQLite adatbázisba. Ezzel lényegesen könnyebben kezelhetővé válik az adathalmaz.

In [1]:
import pyodbc
import pandas as pd
import numpy as np
import os
import sqlite3
from datetime import datetime, timezone

## 1. methods definitions

In [2]:
def extractPeriod(filename):
    """read the period (YYYYMM) value from filename"""
    period = ""
    for char in filename:
        if char.isnumeric():
            period = period + char
    return period

In [3]:
def openListOfFiles():
    """returns a list with the names of the already scanned files"""
    fileListText = open("files_list.csv", "r")
    fileList = fileListText.read()
    fileListText.close()
    print("\tThe list of processed files is loaded")
    return fileList

In [4]:
def writeFilesList(nowProcessed):
    """write the list of now scanned files"""
    fileListText = open("files_list.csv", 'a')
    for step in nowProcessed:
        fileListText.write(step+";")
    fileListText.close()
    print("\tThe nowProcessed filename list writed to file.")

In [5]:
def dirReading(fileList):
    """returns a list with the names of files that have not yet been processed"""
    entries = os.scandir('.')
    toDoList = []
    for entry in entries:
        if entry.is_file():
            filename = entry.name
            if filename.endswith(".mdb"):
                if filename in fileList:
                    pass
                else:
                    toDoList.append(filename)
    print("\tDirectory reading ready", toDoList)
    return toDoList

In [6]:
def mdbReader(path, filename, q):
    """return a DataFrame readed from Access database"""
    file = path+"\\"+filename
    period = extractPeriod(filename)
    if int(period) < 200801:
        query = q + "_" + period
    else:
        query = q
    print(period, "query:", query)
    conn = pyodbc.connect(
        r'Driver={Microsoft Access Driver (*.mdb, *.accdb)};DBQ='+file)
    dfr = pd.read_sql(query, conn)
    dfr.shape
    conn.close()
    print("\tMS-Access file reading OK")
    return dfr

In [7]:
def mdbMetaData(filename, startTime):
    """returns with the list of metadata of selected file"""
    entry = []
    entry.append(startTime)
    statinfo = os.stat(filename)
    entry.append(filename)
    period = extractPeriod(filename)
    year = period[:4]
    entry.append(year)
    entry.append(period)
    entry.append(statinfo.st_size)
    created = datetime.fromtimestamp(statinfo.st_ctime, tz=timezone.utc).strftime("%Y/%m/%d, %H:%M:%S")
    entry.append(created)
    return entry

In [8]:
# https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=web&cd=&ved=2ahUKEwjki_OS-5v5AhVN2aQKHajIDVkQFnoECBEQAQ&url=https%3A%2F%2Fwww.ksh.hu%2Fdocs%2Fhun%2Fxftp%2Froviditesek.pdf&usg=AOvVaw2_-SdIQNO8KCskUhnIm9gd
# módszertan: KSH módszertani útmutató(PDF)
# ("Az ábrázolhatóság érdekében a megyenevek helyett
# egyes grafikonokon az alábbi rövidítéseket alkalmaztuk: ...")

megyedict = {
1:"BA",
2:"BÁ",
3:"BÉ",
4:"BO",
5:"CS",
6:"FE",
7:"GY",
8:"HA",
9:"HE",
10:"KO",
11:"NÓ",
12:"PE",
13:"SO",
14:"SZ",
15:"JÁ",
16:"TO",
17:"VA",
18:"VE",
19:"ZA",
20:"BP",
21:"BP",
'Bács-Kiskun':'BÁ',
'Baranya':'BA',
'Békés':'BÉ',
'Borsod-Abaúj-Zemplén':'BO',
'Budapest':'BP',
'Csongrád-Csanád': 'CS',
'Csongrád':'CS',
'Fejér':'FE',
'Fejé':'FE',
'Győr-Moson-Sopron':'GY',
'Hajdú-Bihar':'HA',
'Heves':'HE',
'Jász-Nagykun-Szolnok':'JÁ',
'Komárom-Esztergom':'KO',
'Nógrád':'NÓ',
'Pest':'PE',
'Somogy':'SO',
'Szabolcs-Szatmár-Bereg':'SZ',
'Tolna':'TO',
'Vas':'VA',
'Veszprém':'VE',
'Zala':'ZA'}

In [9]:
def mdbDecorator(mdb, step):
    if int(extractPeriod(step)) < 200801:
        mdb.columns=['idoszak', 'TTT', 'nev', 'kisznev', 'tk', 'atc', 'jogcim', 'megye', 'doboz', 'tbtam', 'fogyar', 'terdij', 'kvater']
        mdb.insert(1, 'brand', np.nan)
        mdb.insert(2, 'brandid', np.nan)
        mdb.insert(8, 'hatoanyag', np.nan)
        mdb.insert(9, 'venytipus', np.nan)
        mdb.insert(13, 'dotforg', np.nan)
    elif int(extractPeriod(step)) < 200802:
        mdb.columns=['idoszak', 'Megyekód', 'TTT', 'nev', 'kisznev', 'tk', 'atc', 'jogcim', 'doboz', 'tbtam', 'fogyar', 'terdij', 'kvater']
        mdb.insert(1, 'brand', np.nan)
        mdb.insert(2, 'brandid', np.nan)
        mdb.insert(9, 'hatoanyag', np.nan)
        mdb.insert(10, 'venytipus', np.nan)
        mdb.insert(12, 'megye', np.nan)
        mdb['megye'] = mdb['Megyekód']
        del mdb['Megyekód']
        mdb.insert(13, 'dotforg', np.nan)
    elif int(extractPeriod(step)) < 200901:
        mdb.insert(10, 'venytipus', np.nan)
        mdb.columns = mdb.columns.str.lower()
    else:
        mdb.columns=['idoszak', 'brand', 'brandid', 'TTT', 'nev', 'kisznev', 'tk', 'atc', 'hatoanyag','venytipus', 'jogcim', 'megye', 'doboz','dotforg', 'tbtam', 'fogyar', 'terdij', 'kvater']
    return mdb

In [10]:
def priceClassifier(fogyar, doboz):
    if doboz == 0:
        price = 0
    else:
        price = abs(fogyar/doboz)
    if price < 1000:
        return "A:1000 Ft"
    elif price < 2000:
        return "B:1-2 ezer Ft"
    elif price < 4000:
        return "C:2-4 ezer Ft"
    elif price < 6000:
        return "D:4-6 ezer Ft"
    elif price < 10000:
        return "E:6-10 ezer Ft"
    elif price < 20000:
        return "F:10-20 ezer Ft"
    elif price < 40000:
        return "G:20-40 ezer Ft"
    elif price < 150000:
        return "I:90-150 ezer Ft"
    elif price < 250000:
        return "J:150-250 ezer Ft"
    elif price < 500000:
        return "K:250-500 ezer Ft"
    else:
        return "L:500 ezer Ft felett"

In [11]:
def copaymentClassifier(terdij, doboz):
    if doboz == 0:
        copayment = 0
    else:
        copayment = abs(terdij/doboz)
    if copayment < 500:
        return "1:500 Ft alatt"
    elif copayment < 1000:
        return "2:500-1000 Ft"
    elif copayment < 2000:
        return "3:1000-2000 Ft"
    elif copayment < 3000:
        return "4:2000-3000 Ft"
    elif copayment < 5000:
        return "5:3000-5000 Ft"
    elif copayment < 10000:
        return "6:5000-10000 Ft"
    elif copayment < 25000:
        return "7:10000-25000 Ft"
    elif copayment < 75000:
        return "8:25000-75000 Ft"
    elif copayment < 150000:
        return "9:75000-150000 Ft"
    elif copayment < 1000000:
        return "10:150000 - 1000000 Ft"
    else:
        return "11:1000000 Ft felett"

## 2. the loader method

In [12]:
# declarations:

query = "SELECT * FROM Megyei_forgalom"
directory = r"C:\Users\Emcsike\Desktop\TOM\_vényforgalmi_v2\neak_source"
journal = []
nowProcessed = []

In [13]:
# main loader process

db = sqlite3.connect('neak2.sqlite3')
journal = []
nowProcessed = []
alreadyProcessed = openListOfFiles()
toDoList = dirReading(alreadyProcessed)
startTime = datetime.now().strftime("%y-%m-%d_%H-%M-%S")

for step in toDoList:
    mdb = mdbReader(directory, step, query)
    entry = mdbMetaData(step, startTime)
    entry.append(len(mdb))
    mdb = mdbDecorator(mdb, step)
    mdb['megye'] = mdb['megye'].map(megyedict)
    entry.append(mdb['doboz'].sum()/1000000) 
    entry.append(mdb['fogyar'].sum()/1000000000)
    mdb['priceClass'] = mdb.apply(lambda mdb: priceClassifier(mdb['fogyar'], mdb['doboz']), axis=1)
    mdb['copaymClass'] = mdb.apply(lambda mdb: copaymentClassifier(mdb['terdij'], mdb['doboz']), axis=1)
    year = extractPeriod(step)[:4]
    mdb.to_sql(("whole"+year), db, if_exists = 'append')
    nowProcessed.append(step)
    journal.append(entry)
db.close()
print("ready")
writeFilesList(nowProcessed)
dfprocessed = pd.DataFrame(journal, columns =['startTime', 'filename', 'year', 'period', 'size', 'created', 'sourceRecordNumber', 'dfSumBoxMill', 'dfSumPriceBill' ])
period = extractPeriod(step)
STR = 'already_processed_log_'+period+'.csv'
dfprocessed.to_csv(STR, sep=";", encoding='utf-8', index=False)

	The list of processed files is loaded
	Directory reading ready ['OEP_GYSZ_Forg_202208.mdb']
202208 query: SELECT * FROM Megyei_forgalom
	MS-Access file reading OK
ready
	The nowProcessed filename list writed to file.


In [14]:
df_already_scanned = pd.read_csv("already_scanned.csv", sep=";")
df_recent = pd.concat([df_already_scanned, dfprocessed]).reset_index(drop = True)

In [15]:
df_recent.to_csv("already_scanned.csv", sep=";", encoding='utf-8', index=False)

## 3. Current year: other calculations

van két view az adatbázisban:
- year_2016-1-8 ez 2016-tól az első 8 hónapot összegzi fel; ebből a view-ből havonta újat kell készíteni 2016-1-9, stb
- years ez 2006 óta az éveket összegzi fel

Ennek a két view-nek az adatait kell beolvasni és az adatokat két célre használni:
- 16 év áttekintéséhez ("Élet az Excel után" elemzésbe): 2006 mint bázisév után a változásokat kell kiszámolni évről évre fogyár és dobozforgalom szerint
- 7 évnyi áttekintéshez az időarányos adatokat kell az éves adatokhoz fésülni, és időszaki megvalósulás arányokat mutatni

In [12]:
# a beolvasásnál készített already_scanned.csv file visszaolvasása

df_recent = pd.read_csv("already_scanned.csv", sep=";")
df_recent

,startTime,filename,year,period,size,created,sourceRecordNumber,dfSumBoxMill,dfSumPriceBill
0,22-07-11_17-38-47,200601.mdb,2006,200601,36003840,"2022/07/11, 15:37:59",75334,18.521485,38.657073
1,22-07-11_17-38-47,200602.mdb,2006,200602,36048896,"2022/07/11, 15:37:59",75164,17.655448,37.332308
2,22-07-11_17-38-47,200603.mdb,2006,200603,36667392,"2022/07/11, 15:37:59",76627,19.989567,42.924976
3,22-07-11_17-38-47,200604.mdb,2006,200604,36282368,"2022/07/11, 15:38:00",75955,17.563378,38.227791
4,22-07-11_17-38-47,200605.mdb,2006,200605,36950016,"2022/07/11, 15:38:00",77285,19.693307,43.384919
...,...,...,...,...,...,...,...,...,...
195,22-07-11_19-09-57,OEP_GYSZ_Forg_202204.mdb,2022,202204,36458496,"2022/07/11, 17:09:18",116689,15.557308,47.674729
196,22-07-11_19-09-57,OEP_GYSZ_Forg_202205.mdb,2022,202205,36782080,"2022/07/11, 17:09:19",117621,16.322785,50.535769
197,22-07-27_18-07-15,OEP_GYSZ_Forg_202206.mdb,2022,202206,36417536,"2022/07/27, 12:21:48",116591,16.050452,50.421442
198,22-08-29_11-36-44,OEP_GYSZ_Forg_202207.mdb,2022,202207,36020224,"2022/08/26, 11:49:06",115056,14.974588,47.532797


In [13]:
db = sqlite3.connect('neak2.sqlite3')

In [14]:
# az éves felösszegzés első beolvasása
q = "select * from years"
df_years = pd.read_sql(q, db)
sumdob2006 = df_years.iloc[0,1]
sumtb2006 = df_years.iloc[0,2]
sumfogyar2006 = df_years.iloc[0,3]
df_years['sumdob2006']=sumdob2006
df_years['sumtb2006']=sumtb2006
df_years['sumfogyar2006']=sumfogyar2006
df_years['changeSumDob'] = (((df_years['sumdob']/df_years.shift(1, axis = 0)['sumdob'])-1)*100).round(decimals=2)
df_years['changeSumFogyar'] = (((df_years['sumfogyar']/df_years.shift(1, axis = 0)['sumfogyar'])-1)*100).round(decimals=2)
df_years = df_years.fillna(0)

In [46]:
df_years

,year,sumdob,sumtb,sumfogyar,sumterdij,ttt_db,sumdob2006,sumtb2006,sumfogyar2006,changeSumDob,changeSumFogyar
0,2006,2.242601e+08,3.833114e+11,4.879459e+11,1.046341e+11,4925,2.242601e+08,3.833114e+11,4.879459e+11,0.00,0.00
1,2007,1.967550e+08,3.146855e+11,4.338899e+11,1.192047e+11,4750,2.242601e+08,3.833114e+11,4.879459e+11,-12.26,-11.08
2,2008,1.890751e+08,3.288048e+11,4.419597e+11,1.131452e+11,4372,2.242601e+08,3.833114e+11,4.879459e+11,-3.90,1.86
3,2009,2.114415e+08,3.509138e+11,4.666973e+11,1.157821e+11,5189,2.242601e+08,3.833114e+11,4.879459e+11,11.83,5.60
4,2010,2.038948e+08,3.687428e+11,4.849429e+11,1.162015e+11,5295,2.242601e+08,3.833114e+11,4.879459e+11,-3.57,3.91
5,2011,2.072410e+08,3.817761e+11,5.016522e+11,1.198760e+11,5649,2.242601e+08,3.833114e+11,4.879459e+11,1.64,3.45
6,2012,2.014398e+08,3.166038e+11,4.290594e+11,1.124553e+11,5645,2.242601e+08,3.833114e+11,4.879459e+11,-2.80,-14.47
7,2013,2.001104e+08,2.986417e+11,4.084084e+11,1.097691e+11,5420,2.242601e+08,3.833114e+11,4.879459e+11,-0.66,-4.81
8,2014,2.034786e+08,3.036588e+11,4.158247e+11,1.121658e+11,5324,2.242601e+08,3.833114e+11,4.879459e+11,1.68,1.82
9,2015,2.051281e+08,3.240291e+11,4.409120e+11,1.168424e+11,5275,2.242601e+08,3.833114e+11,4.879459e+11,0.81,6.03


In [45]:
df_years.to_sql(("df_years"), db, if_exists = 'replace')

In [ ]:
# az augusztusok első beolvasása
q = "select * from year201618"
df_aug = pd.read_sql(q, db)

In [102]:
df_aug.rename(columns = {'YEAR':'year'}, inplace = True)

In [103]:
df_aug

,year,sumdob8,sumtb8,sumfogyar8,sumterdij8,ttt_db8
0,2016,1.354375e+08,2.230482e+11,3.026832e+11,7.963482e+10,5050
1,2017,1.360539e+08,2.326854e+11,3.150845e+11,8.239894e+10,5086
2,2018,1.366200e+08,2.460698e+11,3.306135e+11,8.468438e+10,4982
3,2019,1.341312e+08,2.636859e+11,3.495211e+11,8.583264e+10,4974
4,2020,1.345720e+08,2.790105e+11,3.689581e+11,8.994241e+10,4990
5,2021,1.265930e+08,2.811049e+11,3.691733e+11,8.805494e+10,5075
6,2022,1.274939e+08,2.988775e+11,3.939876e+11,9.508988e+10,5175


In [104]:
df_monthly = df_years.tail(7).reset_index(drop=True)

In [105]:
df_monthly

,year,sumdob,sumtb,sumfogyar,sumterdij,ttt_db,sumdob2006,sumtb2006,sumfogyar2006,changeSumDob,changeSumFogyar
0,2016,2.050875e+08,3.407646e+11,4.616961e+11,1.209313e+11,5217,2.242601e+08,3.833114e+11,4.879459e+11,-0.02,4.71
1,2017,2.052538e+08,3.515345e+11,4.760961e+11,1.245613e+11,5222,2.242601e+08,3.833114e+11,4.879459e+11,0.08,3.12
2,2018,2.052067e+08,3.735502e+11,5.008184e+11,1.274085e+11,5118,2.242601e+08,3.833114e+11,4.879459e+11,-0.02,5.19
3,2019,2.027733e+08,4.001562e+11,5.310880e+11,1.309264e+11,5159,2.242601e+08,3.833114e+11,4.879459e+11,-1.19,6.04
4,2020,2.020601e+08,4.229058e+11,5.587157e+11,1.358009e+11,5167,2.242601e+08,3.833114e+11,4.879459e+11,-0.35,5.20
5,2021,1.946226e+08,4.318035e+11,5.685446e+11,1.367181e+11,5300,2.242601e+08,3.833114e+11,4.879459e+11,-3.68,1.76
6,2022,1.274939e+08,2.988775e+11,3.939876e+11,9.508988e+10,5175,2.242601e+08,3.833114e+11,4.879459e+11,-34.49,-30.70


In [106]:
df_monthly = pd.merge(df_monthly, df_aug, on='year')

In [107]:
df_monthly

,year,sumdob,sumtb,sumfogyar,sumterdij,ttt_db,sumdob2006,sumtb2006,sumfogyar2006,changeSumDob,changeSumFogyar,sumdob8,sumtb8,sumfogyar8,sumterdij8,ttt_db8
0,2016,2.050875e+08,3.407646e+11,4.616961e+11,1.209313e+11,5217,2.242601e+08,3.833114e+11,4.879459e+11,-0.02,4.71,1.354375e+08,2.230482e+11,3.026832e+11,7.963482e+10,5050
1,2017,2.052538e+08,3.515345e+11,4.760961e+11,1.245613e+11,5222,2.242601e+08,3.833114e+11,4.879459e+11,0.08,3.12,1.360539e+08,2.326854e+11,3.150845e+11,8.239894e+10,5086
2,2018,2.052067e+08,3.735502e+11,5.008184e+11,1.274085e+11,5118,2.242601e+08,3.833114e+11,4.879459e+11,-0.02,5.19,1.366200e+08,2.460698e+11,3.306135e+11,8.468438e+10,4982
3,2019,2.027733e+08,4.001562e+11,5.310880e+11,1.309264e+11,5159,2.242601e+08,3.833114e+11,4.879459e+11,-1.19,6.04,1.341312e+08,2.636859e+11,3.495211e+11,8.583264e+10,4974
4,2020,2.020601e+08,4.229058e+11,5.587157e+11,1.358009e+11,5167,2.242601e+08,3.833114e+11,4.879459e+11,-0.35,5.20,1.345720e+08,2.790105e+11,3.689581e+11,8.994241e+10,4990
5,2021,1.946226e+08,4.318035e+11,5.685446e+11,1.367181e+11,5300,2.242601e+08,3.833114e+11,4.879459e+11,-3.68,1.76,1.265930e+08,2.811049e+11,3.691733e+11,8.805494e+10,5075
6,2022,1.274939e+08,2.988775e+11,3.939876e+11,9.508988e+10,5175,2.242601e+08,3.833114e+11,4.879459e+11,-34.49,-30.70,1.274939e+08,2.988775e+11,3.939876e+11,9.508988e+10,5175


In [108]:
df_monthly.to_sql(("df_monthly"), db, if_exists = 'replace')

In [ ]:
db.close()